In [ ]:
import torch
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#need to load data
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ROCStories_winter2017 - ROCStories_winter2017.csv")


In [ ]:
pd.set_option('display.max_columns', None)
print(df.head())

                                storyid               storytitle  \
0  8bbe6d11-1e2e-413c-bf81-eaea05f4f1bd   David Drops the Weight   
1  0beabab2-fb49-460e-a6e6-f35a202e3348              Frustration   
2  87da1a22-df0b-410c-b186-439700b70ba6       Marcus Buys Khakis   
3  2d16bcd6-692a-4fc0-8e7c-4a6f81d9efa9       Different Opinions   
4  c71bb23b-7731-4233-8298-76ba6886cee1  Overcoming shortcomings   

                                           sentence1  \
0  David noticed he had put on a lot of weight re...   
1                       Tom had a very short temper.   
2  Marcus needed clothing for a business casual e...   
3  Bobby thought Bill should buy a trailer and ha...   
4          John was a pastor with a very bad memory.   

                                           sentence2  \
0  He examined his habits to try and figure out t...   
1               One day a guest made him very angry.   
2  All of his clothes were either too formal or t...   
3  Bill thought a truck would 

## Storyline Planning (static)
Use BiLSTM for encoding and LSTM for decoding

In [ ]:
!pip install rake-nltk
from rake_nltk import Rake
import nltk

nltk.download('stopwords')
nltk.download('punkt')

from nltk.corpus import stopwords

# Uses stopwords for english from NLTK, and all punctuation characters by
# default
#max_length=1 so that we only get one word
r = Rake(max_length=1, min_length=1, include_repeated_phrases=False)

def rake_implement(s1, s2, s3, s4, s5, r):
    result = []
    result.append(get_word(r, s1))
    result.append(get_word(r, s2))
    result.append(get_word(r, s3))
    result.append(get_word(r, s4))
    result.append(get_word(r, s5))
    return result

def get_word(r, s):
  r.extract_keywords_from_text(s)
  phrases = r.get_ranked_phrases()
  if(len(phrases) == 0):
    #if nothing is extracted, take first word that is not in stopwords (from the middle)
    sentence = s.split()
    for i in range(int(len(sentence)/2), len(sentence)):
      if not sentence[i] in stopwords.words('english'):
        return sentence[i].lower()
  else:
    return phrases[0]

#https://stackoverflow.com/questions/56836477/apply-nltk-rake-to-each-row-in-dataframe
df['train_storylines'] = df.apply(lambda x: rake_implement(x.sentence1, x.sentence2, x.sentence3, x.sentence4, x.sentence5, r), axis=1)

print(df.head())

     |████████████████████████████████| 1.5 MB 5.3 MB/s 
     |████████████████████████████████| 749 kB 21.1 MB/s 
  Attempting uninstall: regex
    Found existing installation: regex 2019.12.20
    Uninstalling regex-2019.12.20:
      Successfully uninstalled regex-2019.12.20
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


                                storyid               storytitle  \
0  8bbe6d11-1e2e-413c-bf81-eaea05f4f1bd   David Drops the Weight   
1  0beabab2-fb49-460e-a6e6-f35a202e3348              Frustration   
2  87da1a22-df0b-410c-b186-439700b70ba6       Marcus Buys Khakis   
3  2d16bcd6-692a-4fc0-8e7c-4a6f81d9efa9       Different Opinions   
4  c71bb23b-7731-4233-8298-76ba6886cee1  Overcoming shortcomings   

                                           sentence1  \
0  David noticed he had put on a lot of weight re...   
1                       Tom had a very short temper.   
2  Marcus needed clothing for a business casual e...   
3  Bobby thought Bill should buy a trailer and ha...   
4          John was a pastor with a very bad memory.   

                                           sentence2  \
0  He examined his habits to try and figure out t...   
1               One day a guest made him very angry.   
2  All of his clothes were either too formal or t...   
3  Bill thought a truck would 

In [ ]:
#to convert words to indices and vice versa
#https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html
class Vocab:
    def __init__(self, name):
        self.name = name
        self.word2index = {"<SOS>": 0, "<EOT>": 1, "<EOS>": 2}
        self.word2count = {}
        self.index2word = {0: "<SOS>", 1:"<EOT>", 2:"<EOS>"}
        self.n_words = 3
    
    def addTitle(self, title):
      for word in title.split():
        self.addWordToCount(word)

    def addStoryline(self, storyline):
        for word in storyline:
            #could do preprocessing here
            self.addWordToCount(word)

    def addSentence(self, sentence):
      for word in sentence.split():
        self.addWordToCount(word)

    def addWordToCount(self, word):
        if word not in self.word2count:
            self.word2count[word] = 1
        else:
            self.word2count[word] += 1

    def addWordToDicts(self, word):
      self.word2index[word] = self.n_words
      self.index2word[self.n_words] = word
      self.n_words += 1

    def removeWordFromCount(self, word):
      self.word2count.pop(word)
    
    def length(self):
      return len(self.index2word)



In [ ]:
def prepareVocab(df):
  vocab = Vocab("trainingVocab")
  for title in df["storytitle"]:
    vocab.addTitle(title)
  for s in df["sentence1"]:
    vocab.addSentence(s)
  for s in df["sentence2"]:
    vocab.addSentence(s)
  for s in df["sentence3"]:
    vocab.addSentence(s)
  for s in df["sentence4"]:
    vocab.addSentence(s)
  for s in df["sentence5"]:
    vocab.addSentence(s)
  for storyline in df["train_storylines"]:
    vocab.addStoryline(storyline)
  
  #go thru counts and combine all those with just 1 count into <unk>
  #unkCount = 0
  #low_dict = {key: value for key, value in vocab.word2count.items() if value == 1}
  #for key, value in low_dict.items():
    #add 1 to unkCount and remove from dict
    #unkCount += 1
    #vocab.removeWordFromCount(key)
  # add unk, then add rest of words to dicts
  #vocab.addWordToDicts("<unk>")
  for key, value in vocab.word2count.items():
    vocab.addWordToDicts(key)
  
  return vocab


In [ ]:
# divide into training, (dev and test)
msk = np.random.rand(len(df)) < 0.01 #reducing training just to test something out (0.8)
reduced_df = df[msk]
msk = np.random.rand(len(reduced_df)) < 0.8
train_df = reduced_df[msk]
eval_df = reduced_df[~msk]
msk = np.random.rand(len(eval_df)) < 0.5
dev_df = eval_df[msk]
test_df = eval_df[~msk]

In [ ]:
#prepare vocab! Put both training and test?
vocab = prepareVocab(reduced_df)

In [ ]:
INPUT_SIZE = 15
HIDDEN_SIZE = 1000
EMBEDDING_SIZE = 500
OUTPUT_SIZE = vocab.length()
TARGET_SIZE = 5
NUM_LAYERS = 1
BATCH_SIZE = 50
NUM_WORKERS = 0
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# create datasets 
from torch.utils.data import DataLoader, Dataset
class StorylineDataset (Dataset):
    def __init__(self, titles, storylines, vocab, input_size):
        #must convert words to numeric representations
        self.titles = []
        for title in titles:
          temp = []
          for word in title.split():
            if word in vocab.word2index:
              temp.append(vocab.word2index[word])
            else:
              temp.append(vocab.word2index[""])
          #pad
          self.titles.append(np.pad(temp, (input_size-len(temp), 0), 'constant'))
        
        self.storylines = []
        for storyline in storylines:
          temp = []
          for word in storyline:
            if word in vocab.word2index:
              temp.append(vocab.word2index[word])
            else:
              temp.append(vocab.word2index[""])
          self.storylines.append(temp)
        
    def __len__(self):
        return len(self.titles)
    
    def __getitem__(self, index):
        title = np.asarray(self.titles[index]) 
        storyline = np.asarray(self.storylines[index])

        title = torch.from_numpy(title).long()
        storyline = torch.from_numpy(storyline).long()   
        return (title, storyline)

In [ ]:
training_data = StorylineDataset(train_df["storytitle"], train_df["train_storylines"], vocab, INPUT_SIZE)
testing_data = StorylineDataset(test_df["storytitle"], test_df["train_storylines"], vocab, INPUT_SIZE)

In [ ]:
training_loader = torch.utils.data.DataLoader(training_data, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
testing_loader = torch.utils.data.DataLoader(testing_data, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)

In [ ]:
#encoder
class TitleEncoder(torch.nn.Module):
    def __init__(self, input_size, hidden_size, embedding_size, vocab_size, num_layers):
        super(TitleEncoder, self).__init__()
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.embedding_size = embedding_size
        self.vocab_size = vocab_size
        
        #print(vocab_size)
        #print(embedding_size)
        self.embedding = torch.nn.Embedding(vocab_size, embedding_size)
        self.bilstm = torch.nn.LSTM(self.embedding_size, self.hidden_size, num_layers=num_layers, batch_first=True, bidirectional=True)
        self.hidden2hidden = torch.nn.Linear(2*self.hidden_size, self.hidden_size)

    def forward(self, title):
        embedded = self.embedding(title)
        bilstm_out, hidden_out = self.bilstm(embedded)
        #print(hidden_out)
        #print(type(hidden_out))
        output = self.hidden2hidden(bilstm_out)
        return output, hidden_out


In [ ]:
#decoder
# need to incorporate attention: https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html
#initial input token is the start of string token, and the first hidden state si the context vector
class StorylineDecoder(torch.nn.Module):
    def __init__(self, hidden_size, output_size, embedding_size, num_layers, e_dropout_p, h_dropout_p=0.1):
        super(StorylineDecoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding_size = embedding_size
        self.output_size = output_size
        self.e_dropout_p = e_dropout_p
        self.h_dropout_p = h_dropout_p

        self.embedding = torch.nn.Embedding(self.output_size, self.embedding_size)
        self.lstm = torch.nn.LSTM(self.embedding_size, self.hidden_size, num_layers=num_layers, batch_first=True)
        self.out = torch.nn.Linear(self.hidden_size, self.output_size)
        #self.softmax = torch.nn.LogSoftmax(dim=1)
        self.e_dropout = torch.nn.Dropout(self.e_dropout_p)
        self.h_dropout = torch.nn.Dropout(self.h_dropout_p)       

    def forward(self, input, hidden):
        embedded = self.embedding(input)
        embedded = self.e_dropout(embedded)
        lstm_out, hidden_out = self.lstm(embedded, hidden)
        output = self.out(lstm_out)
        output = self.h_dropout(output)
        #output = self.softmax(output)
        return output, hidden_out

In [ ]:
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, batch_size=BATCH_SIZE, target_length=TARGET_SIZE):
    encoder_hidden = torch.zeros(1, 1, HIDDEN_SIZE, device=device)

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = INPUT_SIZE#input_tensor.size(0)
    #target_length = TARGET_SIZE#target_tensor.size(0)

    #encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    #batch first
    encoder_output, hidden = encoder(input_tensor)
    #encoder_outputs[ei] = encoder_output[0, 0]

    #start with SOS token
    decoder_input = torch.tensor([[0]], device=device)


    use_teacher_forcing = False #True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        #go thru batch first
        for b in range(len(hidden[0][0])):
          h_0 = torch.zeros(1, 1, HIDDEN_SIZE*2, device=device)
          h_0[0][0] = hidden[0].transpose(0,1)[b].flatten()
          c_0 = torch.zeros(1, 1, HIDDEN_SIZE*2, device=device)
          c_0[0][0] = hidden[1].transpose(0,1)[b].flatten()
          decoder_hidden = (h_0, c_0)

          for di in range(target_length):
              decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
              topv, topi = decoder_output.topk(1)
              decoder_input = topi.squeeze().detach()  # detach from history as input
              decoder_input = torch.tensor([[decoder_input]], device=device)
              #print(target_tensor)
              reshaped_target = torch.zeros(1, device=device).long()
              reshaped_target[0] = target_tensor[b][di]
              #print(target_tensor[b][di])
              #print(reshaped_target)
              #print(vocab.index2word[topi[0][0][0].item()])
              #print(vocab.index2word[reshaped_target[0].item()])
              #print(decoder_output)
              #print(decoder_output[0])
              loss += criterion(decoder_output[0], reshaped_target)

    #print("backwards step")
    loss.backward()
    print("loss: ", loss.item() / (len(hidden[0][0])*target_length))

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / (len(hidden[0][0])*target_length)

In [ ]:
#init models
encoder_model = TitleEncoder(INPUT_SIZE, HIDDEN_SIZE, EMBEDDING_SIZE, vocab.length(), NUM_LAYERS).to(device)
decoder_model = StorylineDecoder(HIDDEN_SIZE*2, OUTPUT_SIZE, EMBEDDING_SIZE, NUM_LAYERS, 0.4).to(device)

In [ ]:
#load encoder_model
encoder_model.load_state_dict(torch.load("/content/drive/MyDrive/Colab Notebooks/encoder_title.pt"))

In [ ]:
#load decoder_model
decoder_model.load_state_dict(torch.load("/content/drive/MyDrive/Colab Notebooks/decoder_storyline.pt"))

In [ ]:
# create criterion and optimizer
criterion = torch.nn.CrossEntropyLoss()
encoder_optimizer = torch.optim.SGD(encoder_model.parameters(), lr = 0.01, momentum=0.9)#torch.optim.Adam(encoder_model.parameters(), lr=0.01) #torch.optim.SGD(encoder_model.parameters(), lr = 0.01, momentum=0.9)
decoder_optimizer = torch.optim.SGD(decoder_model.parameters(), lr = 0.01, momentum=0.9)#torch.optim.Adam(decoder_model.parameters(), lr=0.01) #torch.optim.SGD(decoder_model.parameters(), lr = 0.01, momentum=0.9)
#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.9)

In [ ]:
def trainModel(num_epochs, train_loader, test_loader, encoder_model, decoder_model, encoder_optimizer, decoder_optimizer, criterion):
  #keep track of validation loss
  valid_loss_min = float('inf')
  for epoch in range(1, num_epochs+1):
            total_loss = 0.0
            #put models in train mode
            encoder_model.train()
            decoder_model.train()
            for titles, storylines in train_loader:
              #send to cuda
              titles.to(device)
              storylines.to(device)
              #call train function
              total_loss += train(titles, storylines, encoder_model, decoder_model, encoder_optimizer, decoder_optimizer, criterion)
            
            print("Epoch ", epoch, " training loss: ", total_loss/len(train_loader))

In [ ]:
# call trainModel 0.047
trainModel(3, training_loader, testing_loader, encoder_model, decoder_model, encoder_optimizer, decoder_optimizer, criterion)

loss:  20.149923828125
loss:  24.896830078125
loss:  17.19998828125
loss:  25.257783203125
loss:  25.016263671875
loss:  24.500158203125
loss:  20.377439453125
loss:  27.589599609375
loss:  19.265682547433034
Epoch  1  training loss:  22.694852097284226
loss:  17.20325390625
loss:  19.303689453125
loss:  16.496734375
loss:  19.777203125
loss:  18.242998046875
loss:  25.30678125
loss:  18.636916015625
loss:  20.6531484375
loss:  21.169011579241072
Epoch  2  training loss:  19.64330402095734
loss:  17.0391640625
loss:  16.0817314453125
loss:  12.9033359375
loss:  15.1892333984375
loss:  16.36453515625
loss:  14.4716396484375
loss:  16.3831845703125
loss:  13.1266572265625
loss:  12.277472795758928
Epoch  3  training loss:  14.87077269345238


In [ ]:
#save encoder_model
torch.save(encoder_model.state_dict(), "/content/drive/MyDrive/Colab Notebooks/encoder_title.pt")

In [ ]:
#save decoder_model
torch.save(encoder_model.state_dict(), "/content/drive/MyDrive/Colab Notebooks/decoder_storyline.pt")

In [ ]:
def evaluate(input_tensor, target_tensor, encoder, decoder, criterion, batch_size=BATCH_SIZE, target_length=TARGET_SIZE):
  with torch.no_grad():
    encoder_hidden = torch.zeros(1, 1, HIDDEN_SIZE, device=device)

    input_length = INPUT_SIZE#input_tensor.size(0)
    #target_length = TARGET_SIZE#target_tensor.size(0)

    loss = 0

    #batch first
    encoder_output, hidden = encoder(input_tensor)
    #encoder_outputs[ei] = encoder_output[0, 0]

    #start with SOS token
    decoder_input = torch.tensor([[0]], device=device)

    titles = []
    predicteds = []
    targets = []


    # Without teacher forcing: use its own predictions as the next input
    #go thru batch first
    for b in range(len(hidden[0][0])):
      title = []
      predicted = []
      target = []
      for i in range(len(input_tensor[b])):
        title.append(vocab.index2word[input_tensor[b][i].item()])

      h_0 = torch.zeros(1, 1, HIDDEN_SIZE*2, device=device)
      h_0[0][0] = hidden[0].transpose(0,1)[b].flatten()
      c_0 = torch.zeros(1, 1, HIDDEN_SIZE*2, device=device)
      c_0[0][0] = hidden[1].transpose(0,1)[b].flatten()
      decoder_hidden = (h_0, c_0)
      
      for di in range(target_length):
          decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
          topv, topi = decoder_output.topk(1)
          decoder_input = topi.squeeze().detach()  # detach from history as input
          decoder_input = torch.tensor([[decoder_input]], device=device)
          #print(target_tensor)
          reshaped_target = torch.zeros(1, device=device).long()
          reshaped_target[0] = target_tensor[b][di]
          loss += criterion(decoder_output[0], reshaped_target)

          predicted.append(vocab.index2word[topi[0][0][0].item()])
          target.append(vocab.index2word[reshaped_target[0].item()])
          
      if (b == 0 or b == len(hidden[0][0])-1):
        print("title: ", title)
        print("predicted: ", predicted)
        print("Target: ", target)
        print('\n')
      #append to output lists
      titles.append(title)
      predicteds.append(predicted)
      targets.append(target)

    return (loss.item() / (len(hidden[0][0])*target_length), titles, predicteds, targets)

In [ ]:
def evaluateModel(test_loader, encoder_model, decoder_model, criterion):
  #keep track of validation loss
  valid_loss_min = float('inf')
  total_loss = 0.0
  for titles, storylines in test_loader:
    #send to cuda
    titles.to(device)
    storylines.to(device)
    #call train function
    loss, titles, predicteds, targets = evaluate(titles, storylines, encoder_model, decoder_model, criterion)
    total_loss += loss
  print("total loss: ", total_loss/len(test_loader))

In [ ]:
evaluateModel(testing_loader, encoder_model, decoder_model, criterion)

title:  ['<SOS>', '<SOS>', '<SOS>', '<SOS>', '<SOS>', '<SOS>', '<SOS>', '<SOS>', '<SOS>', '<SOS>', '<SOS>', '<SOS>', 'Mary', "isn't", 'home']
predicted:  ['party', 'wanted', 'home', 'party', 'car']
Target:  ['mary', 'mall', 'running', 'pulled', 'knocks']


title:  ['<SOS>', '<SOS>', '<SOS>', '<SOS>', '<SOS>', '<SOS>', '<SOS>', '<SOS>', '<SOS>', '<SOS>', '<SOS>', '<SOS>', '<SOS>', 'No', 'Help']
predicted:  ['party', 'car', 'go', 'car', 'unfortunately']
Target:  ['solving', 'help', 'wanted', 'hour', 'solved']


title:  ['<SOS>', '<SOS>', '<SOS>', '<SOS>', '<SOS>', '<SOS>', '<SOS>', '<SOS>', '<SOS>', '<SOS>', '<SOS>', '<SOS>', 'The', 'Long', 'Jump']
predicted:  ['party', 'car', 'go', 'car', 'unfortunately']
Target:  ['jose', 'ready', 'began', 'track', 'crashed']


title:  ['<SOS>', '<SOS>', '<SOS>', '<SOS>', '<SOS>', '<SOS>', '<SOS>', '<SOS>', '<SOS>', '<SOS>', '<SOS>', '<SOS>', 'The', 'Wrong', 'Brand']
predicted:  ['party', 'go', 'car', 'unfortunately', 'unfortunately']
Target:  ['nate',

In [ ]:
# create datasets 
from torch.utils.data import DataLoader, Dataset
class SentencesDataset (Dataset):
    def __init__(self, titles, storylines, sentence1, sentence2, sentence3, sentence4, sentence5, vocab, input_size, output_size):
        #must convert words to numeric representations
        #convert all into separate arrays first
        self.titles = []
        for title in titles:
          temp = []
          for word in title.split():
            if word in vocab.word2index:
              temp.append(vocab.word2index[word])
            else:
              temp.append(vocab.word2index[""])
          #pad
          self.titles.append(temp)
        
        self.storylines = []
        for storyline in storylines:
          temp = []
          for word in storyline:
            if word in vocab.word2index:
              temp.append(vocab.word2index[word])
            else:
              temp.append(vocab.word2index[""])
          self.storylines.append(temp)

        #convert titles and storylines to combined_input
        self.combined_input = []
        for i in range(len(self.titles)):
          temp = []
          temp.extend(self.titles[i])
          temp.append(vocab.word2index['<EOT>'])
          temp.extend(self.storylines[i])
          #pad to input_size amount
          self.combined_input.append(np.pad(temp, (input_size-len(temp), 0), 'constant'))

        self.sentences = []
        for i in range(len(sentence1)):
          temp = []
          for word in sentence1.iloc[i].split():
            if word in vocab.word2index:
              temp.append(vocab.word2index[word])
            else:
              temp.append(vocab.word2index[""])
          temp.append(vocab.word2index["<EOS>"])
          for word in sentence2.iloc[i].split():
            if word in vocab.word2index:
              temp.append(vocab.word2index[word])
            else:
              temp.append(vocab.word2index[""])
          temp.append(vocab.word2index["<EOS>"])
          for word in sentence3.iloc[i].split():
            if word in vocab.word2index:
              temp.append(vocab.word2index[word])
            else:
              temp.append(vocab.word2index[""])
          temp.append(vocab.word2index["<EOS>"])
          for word in sentence4.iloc[i].split():
            if word in vocab.word2index:
              temp.append(vocab.word2index[word])
            else:
              temp.append(vocab.word2index[""])
          temp.append(vocab.word2index["<EOS>"])
          for word in sentence5.iloc[i].split():
            if word in vocab.word2index:
              temp.append(vocab.word2index[word])
            else:
              temp.append(vocab.word2index[""])
          temp.append(vocab.word2index["<EOS>"])
          #append full story to sentences, making sure to pad
          self.sentences.append(np.pad(temp, (0, output_size-len(temp)), 'constant'))
        
    def __len__(self):
        return len(self.titles)
    
    def __getitem__(self, index):
        title = np.asarray(self.titles[index]) 
        storyline = np.asarray(self.storylines[index])
        input = np.asarray(self.combined_input[index])
        story = np.asarray(self.sentences[index])

        #title = torch.from_numpy(title).long()
        #storyline = torch.from_numpy(storyline).long()
        input = torch.from_numpy(input).long()   
        story = torch.from_numpy(story).long()
        return (input, story)

In [ ]:
COMBINED_INPUT_SIZE = 20
STORY_OUTPUT_SIZE = 100

In [ ]:
training_data_story = SentencesDataset(train_df["storytitle"], train_df["train_storylines"], train_df["sentence1"], train_df["sentence2"], train_df["sentence3"], train_df["sentence4"], train_df["sentence5"], vocab, COMBINED_INPUT_SIZE, STORY_OUTPUT_SIZE)
testing_data_story = SentencesDataset(test_df["storytitle"], test_df["train_storylines"], test_df["sentence1"], test_df["sentence2"], test_df["sentence3"], test_df["sentence4"], test_df["sentence5"], vocab, COMBINED_INPUT_SIZE, STORY_OUTPUT_SIZE)

In [ ]:
training_loader_story = torch.utils.data.DataLoader(training_data_story, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
testing_loader_story = torch.utils.data.DataLoader(testing_data_story, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)

In [ ]:
#can we use previous model functions with different parameters?
#init models
encoder_model_story = TitleEncoder(COMBINED_INPUT_SIZE, HIDDEN_SIZE, EMBEDDING_SIZE, vocab.length(), NUM_LAYERS).to(device)
decoder_model_story = StorylineDecoder(HIDDEN_SIZE*2, OUTPUT_SIZE, EMBEDDING_SIZE, NUM_LAYERS, 0.2).to(device)

In [ ]:
# create criterion and optimizer
criterion_story = torch.nn.CrossEntropyLoss() #torch.nn.NLLLoss()
encoder_optimizer_story = torch.optim.SGD(encoder_model.parameters(), lr = 0.01, momentum=0.9)
decoder_optimizer_story = torch.optim.SGD(decoder_model.parameters(), lr = 0.01, momentum=0.9)
#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.9)

In [ ]:
def trainModelStory(num_epochs, train_loader, test_loader, encoder_model, decoder_model, encoder_optimizer, decoder_optimizer, criterion):
  #keep track of validation loss
  valid_loss_min = float('inf')
  for epoch in range(1, num_epochs+1):
            total_loss = 0.0
            #put models in train mode
            encoder_model.train()
            decoder_model.train()
            for inputs, stories in train_loader:
              #send to cuda
              inputs.to(device)
              stories.to(device)
              #call train function
              total_loss += train(inputs, stories, encoder_model, decoder_model, encoder_optimizer, decoder_optimizer, criterion, target_length=STORY_OUTPUT_SIZE)
              #break #for testing purposes only
            print("Epoch ", epoch, " training loss: ", total_loss/len(train_loader))

In [ ]:
# call trainModel
print(vocab.length())
trainModelStory(1, training_loader_story, testing_loader_story, encoder_model_story, decoder_model_story, encoder_optimizer_story, decoder_optimizer_story, criterion_story)

5643


KeyboardInterrupt: ignored

In [ ]:
#save encoder_model
torch.save(encoder_model_story.state_dict(), "/content/drive/MyDrive/Colab Notebooks/encoder_combined.pt")

In [ ]:
#save decoder_model
torch.save(decoder_model_story.state_dict(), "/content/drive/MyDrive/Colab Notebooks/decoder_story.pt")

In [ ]:
def evaluateModelStory(test_loader, encoder_model, decoder_model, criterion):
  #keep track of validation loss
  valid_loss_min = float('inf')
  total_loss = 0.0
  for titles, storylines in test_loader:
    #send to cuda
    titles.to(device)
    storylines.to(device)
    #call train function
    loss, titles, predicteds, targets = evaluate(titles, storylines, encoder_model, decoder_model, criterion, target_length=STORY_OUTPUT_SIZE)
    total_loss += loss
  print("total loss: ", total_loss/len(test_loader))

In [ ]:
evaluateModelStory(testing_loader_story, encoder_model_story, decoder_model_story, criterion_story)

title:  ['<SOS>', '<SOS>', '<SOS>', '<SOS>', '<SOS>', '<SOS>', '<SOS>', '<SOS>', '<SOS>', '<SOS>', '<SOS>', '<SOS>', 'The', 'Solo', '<EOT>', 'solo', 'blew', 'time', 'gave', 'took']
predicted:  ['divorce', 'house!', 'house!', 'push.', 'negotiating', 'wetsuit.', 'bleachers.', "world's", "billy's", 'Margaret', 'following', 'application', 'old.', 'Tally', "He's", 'ecology', 'Whopper', 'wall,', 'theater.', 'test,', 'stomach', 'Thieving', 'water', 'sports', 'argument.', 'like', 'rather', 'rather', 'stock', 'horseshoe', 'should', 'removed,', 'fussed', 'time', 'glass.', 'names', 'names', 'cheating', 'names', 'names', 'cheating', 'emailed', 'numbers', 'restaurant.', 'hoop.', 'hoop.', 'Afterwards,', 'aimlessly', 'deb', 'knit', 'he', 'Student', 'sad.', 'Tea', 'countless', 'declined.', "Year's", 'more', 'before', 'spicy.', 'steps', 'presented', 'too.', 'within', 'paint', 'gas', 'match', 'humvee', 'insisted', 'realtor.', 'yet', 'King.', 'lugged', 'parlor.', 'river,', 'tom', 'parents,', 'Table', 'ja